### This was run in Colab since it consumes a lot of ram

In [1]:
!pip install ultralytics
!pip install roboflow
!pip install supervision

In [2]:
from google.colab import drive
from pathlib import Path
import os
import json
drive.mount('/content/drive')
x = '/content/drive/MyDrive/Football-Analysis-System/'
path = Path(x)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['clips',
 'yolov8x.pt',
 'runs',
 'training',
 'config.json',
 '.ipynb_checkpoints',
 'yolov8n.pt',
 'yolov5lu.pt',
 'yolo_inference.ipynb',
 'video_utils_trials.ipynb',
 'trackers',
 'stubs']

In [3]:
cd  /content/drive/MyDrive/Football-Analysis-System

/content/drive/MyDrive/Football-Analysis-System


In [4]:
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle

Code for video_utils.py

In [5]:
def read_video(video_path):
    """read_video : reads a video file and returns the frames

    Args:
        video_path : frames of the video
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    return frames


def save_video(output_video_frames, output_video_path):
    # XVID is the output format
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    # generate a video at the path, of type fourcc @ 24 frames/second and framesize as
    # the last parameter (width, height)
    out = cv2.VideoWriter(output_video_path, fourcc, 24,
                          (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release

Code for bbox_utils.py

In [6]:
def get_center(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2), int((y1+y2)/2)

def get_width(bbox):
    x1,y1,x2,y2 = bbox
    return int(x2-x1)

def get_height(bbox):
    x1,y1,x2,y2 = bbox
    return int(y2-y1)

Code for tracker.py

In [7]:
class Tracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack()

    def detect_frames(self, frames):
        batch_size =20
        detections = []

        # Split frames into batches of size batch_size
        # and detect each batch separately
        # for faster processing and avoid memory issues

        for i in range(0, len(frames), batch_size):
            batch = frames[i:i+batch_size]
            detections_batch = self.model.predict(batch, conf=0.1)
            detections += detections_batch

        return detections

    def get_object_tracks(self, frames, read_from_stub = False, stub_path = None):

        if read_from_stub and stub_path is not None and os.path.exists(stub_path):
            with open(stub_path, 'rb') as f:
                tracks = pickle.load(f)
            return tracks

        detections = self.detect_frames(frames)
        tracks = {
            'players':[],
            'referees':[],
            'ball':[]
        }

        for frame_num, detection in enumerate(detections):
            class_names = detection.names
            class_names_inv = {i:j for j, i in class_names.items()}

            #convert the detection to supervison format
            detection_sv = sv.Detections.from_ultralytics(detection)

            #convert goalkeeper to player object
            for index, class_id in enumerate(detection_sv.class_id):
                if class_names[class_id] == 'goalkeeper':
                    detection_sv.class_id[index] = class_names_inv['player']

            # Track objects
            detections_with_tracks = self.tracker.update_with_detections(detection_sv)

            tracks['players'].append({})
            tracks['referees'].append({})
            tracks['ball'].append({})

            # Tracking the players and the referee since they are more than one.
            # No need to track the ball using the detection with tracks since there is
            # only one ball in the video hence only one bounding ball
            for frame_detection in detections_with_tracks:
                bbox = frame_detection[0].tolist()
                class_id = frame_detection[3]
                track_id = frame_detection[4]

                if class_id == class_names_inv['player']:
                    tracks['players'][frame_num][track_id] = {"bbox":bbox}
                elif class_id == class_names_inv['referee']:
                    tracks['referees'][frame_num][track_id] = {"bbox":bbox}

            # Tracking the ball...
            for frame_detection in detection_sv:
                bbox = frame_detection[0].tolist()
                class_id = frame_detection[3]

                if class_id == class_names_inv['ball']:
                    # We're using a constant one since there's only one ball
                    tracks['ball'][frame_num][1] = {"bbox":bbox}

            if stub_path is not None:
                with open(stub_path, 'wb') as f:
                    pickle.dump(tracks, f)

            return tracks

    def draw_ellipse(self, frame, bbox, color, track_id=None):
        # Ellipse to be postioned at the bottom of the bounding box
        y2 = int(bbox[3])

        x_center, y_center = get_center(bbox)
        width = get_width(bbox)

        cv2.ellipse(frame,
                    center = (x_center, y2),
                    axes = (int(width), int(0.35*width)),
                    angle = 0.0,
                    startAngle=-45,
                    endAngle=235,
                    color = color,
                    thickness=2,
                    lineType=cv2.LINE_4)

        rectangle_width = 40
        rectangle_height = 20
        x1_rect = x_center - rectangle_width//2
        x2_rect = x_center + rectangle_width//2
        y1_rect = y2 - rectangle_height//2 + 15
        y2_rect = y2 + rectangle_height//2 + 15

        if track_id is not None:
            cv2.rectangle(frame,
                        pt1 = (int(x1_rect), int(y1_rect)),
                        pt2 = (int(x2_rect), int(y2_rect)),
                        color = color,
                        thickness = cv2.FILLED)

            x1_text = int(x1_rect) + 12
            if track_id > 99:
                x1_text -= 10
            y1_text = int(y1_rect) + 15
            cv2.putText(frame,
                        text = f'{track_id}',
                        org = (x1_text, y1_text),
                        fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale = 0.6,
                        color = (0,0,0),
                        thickness = 2
                        )

        return frame

    def draw_triangle(self, frame, bbox, color):
        y = bbox[1]
        x,_ = get_center(bbox)

        triangle_points = np.array([
            [x,y],
            [x-10,y-20],
            [x+10,y-20]
        ], dtype=np.int32)

        # Ensure the triangle_points array is not empty and is a numpy array
        if triangle_points is not None and len(triangle_points) > 0:
            # Reshape the array to the expected structure for drawContours
            triangle_points = triangle_points.reshape((-1, 1, 2))

            # Draw the filled inverted triangle
            cv2.drawContours(frame, [triangle_points], 0, color, cv2.FILLED)
            # Draw the inverted triangle border
            cv2.drawContours(frame, [triangle_points], 0, (0, 0, 0), 2)

        return frame

    def draw_annotations(self, video_frames, tracks):
        output_video_frames = []
        for frame_num, frame in enumerate(video_frames):
            frame = frame.copy()
            players_dict = tracks['players'][frame_num]
            referees_dict = tracks['referees'][frame_num]
            ball_dict = tracks['ball'][frame_num]

            # Draw Players
            for track_id, player in players_dict.items():
                frame = self.draw_ellipse(frame, player['bbox'], (0, 0, 255), track_id)

            # Draw Referee
            for _, referee in referees_dict.items():
                frame = self.draw_ellipse(frame, referee['bbox'], (0, 255, 255))

            # Draw Ball
            for _, ball in ball_dict.items():
                frame = self.draw_triangle(frame, ball['bbox'], (0, 255, 0))

            output_video_frames.append(frame)

        return output_video_frames


code for main.py

In [8]:
# Read video
video_frames = read_video('clips/input/08fd33_4.mp4')

# Initilize Tracker
tracker = Tracker('runs/models/best.pt')
# get object tracks
tracks = tracker.get_object_tracks(video_frames, read_from_stub=True, stub_path='./stubs/track_stubs_final.pkl')

# Draw output
# Draw object tracks
output_video_frames = tracker.draw_annotations(video_frames, tracks)

# save video
save_video(output_video_frames, 'clips/output/output_video.avi')

# save video_frame
# Already saved, that's why it's commented out
# save_video(video_frames, 'clips/output/08fd33_4.avi')